In [1]:
from requests_html import HTMLSession
from time import *
from random import randint
import pandas as pd
import numpy as np
import csv
from requests import request
import pandas_datareader.data as web
from datetime import datetime, timedelta, time, date
# import gspread_dataframe as gd
import gspread as gd
from google.auth.transport.requests import AuthorizedSession
from oauth2client.service_account import ServiceAccountCredentials
import asyncio
from requests_html import AsyncHTMLSession

In [2]:
gc = gd.service_account(filename='Seetzzz-1cb93f64d8d7.json')
worksheet = gc.open("Skript_Ticker").worksheet("Options_calc")

Задаем параметры

In [3]:
ticker = 'INTC'
next_ticker_row = '2'

In [4]:
df = web.DataReader(ticker, 'yahoo')
current_price = round(float(df[::-1]['Close'][0]), 2)
print(f'current_price: {current_price}')

min_put = int(current_price - (current_price * 0.2))
print(f'min_put: {min_put}')

max_сall = int(current_price + (current_price * 0.2))
print(f'max_сall: {max_сall}')

current_price: 61.85
min_put: 49
max_сall: 74


In [5]:
k = 0
# парсим все возможные даты в слекте
while k < 1:
    session = AsyncHTMLSession()
    url = await session.get(f'https://www.barchart.com/stocks/quotes/{ticker}/options')
    await url.html.arender(timeout=300)
    url_parser = url.html.xpath('//*[@id="main-content-column"]/div/div[3]/div[1]/div/div[2]/select')
    date_select = url_parser[0].text.split('\n')
    print(len(date_select))

    if len(date_select) > 1:
        k += 1
        print(date_select)
    session.close()


dates = []
puts_bid = []
call_bid = []
current_price_list = []
cost_put_call = []
income_put = []
income_put_year = []
income_call = []
income_call_yaer = []
day_len_list = []
strike_put_list = []
strike_call_list = []
put_plus_call = []
put_plus_call_year = []
ticker_list = []
days_call = []

14
['2021-02-19 (m)', '2021-02-26 (w)', '2021-03-05 (w)', '2021-03-12 (w)', '2021-03-19 (m)', '2021-03-26 (w)', '2021-04-01 (w)', '2021-04-16 (m)', '2021-06-18 (m)', '2021-07-16 (m)', '2021-09-17 (m)', '2022-01-21 (m)', '2022-06-17 (m)', '2023-01-20 (m)']


<ipython-input-5-53fe6a317472>:14: RuntimeWarning: coroutine 'AsyncHTMLSession.close' was never awaited
  session.close()


In [6]:
# парсим путы в пределах до мин порогда для каждой имеющейся даты
for i in range(len(date_select)): #len(date_select)
    session = AsyncHTMLSession()
    print(date_select[i].replace(' (w)', '-w').replace(' (m)', '-m'))
    date_to_pars = date_select[i].replace(' (w)', '-w').replace(' (m)', '-m')

    iteration = 0
    while iteration < 1:
        url_date = await session.get(f'https://www.barchart.com/stocks/quotes/{ticker}/options?expiration={date_to_pars}')
        await url_date.html.arender(timeout=300)
        try:
            check = url_date.html.xpath(f'//*[@id="main-content-column"]/div/div[5]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div/div/ng-transclude/table')
            text_check = check[0].text
        except:
            text_check = ['0']
            pass
        iteration_len = text_check.count('Quote Overview')
#         print('table len: ', text_check.count('Quote Overview'))

        if len(text_check) > 1:
            iteration += 1

    for j in range(iteration_len):
        it = 0
        while it < 2:
            try:
                puts_parser = url_date.html.xpath(f'//*[@id="main-content-column"]/div/div[5]/div[1]/div/div[2]/div[2]/div[2]/div[2]/div/div/ng-transclude/table/tbody/tr[{j+1}]')
                strike_put = (puts_parser[0].text.split('\n'))
                if len(strike_put) >= 1:
                    it += 1

                if float(strike_put[0]) <= current_price and min_put <= float(strike_put[0]):
                    day_len = (datetime.strptime(date_to_pars.replace('-w', '').replace('-m', ''), '%Y-%m-%d')).date() - date.today()
                    day_len_num = int(str(day_len).split(' ')[0])
                    strike_put_list.append(strike_put[0])
                    dates.append(date_to_pars)
                    puts_bid.append(strike_put[2])
                    current_price_list.append(current_price)
                    cost_put_call.append(current_price*200)
                    income_put.append(float(strike_put[2])*100)
                    day_len_list.append(day_len_num)
                    income_put_year.append((((round(float(strike_put[2]), 2)*100)/(current_price*100))/day_len_num)*365)
                    ticker_list.append(ticker)

                else:
                    pass
            except Exception as err:
                print(err)
                print("EEEEEEEEEERRRRRRRROOOOOORRRRR")
                pass
    #
    #

            # =============== парсим и обрабатываем колы ============

            try:
                call_parser = url_date.html.xpath(
                    f'//*[@id="main-content-column"]/div/div[5]/div[1]/div/div[2]/div[1]/div[2]/div[2]/div/div/ng-transclude/table/tbody/tr[{j + 1}]')
                strike_call = (call_parser[0].text.split('\n'))
                if len(strike_call) >= 1:
                    it += 1

                if current_price <= float(strike_call[0]) and float(strike_call[0]) <= max_сall:
                    call_bid.append(strike_call[2])
                    strike_call_list.append(strike_call[0])
                    income_call.append(float(strike_call[2]) * 100)
                    income_call_yaer.append((((round(float(strike_call[2]), 2) * 100) / (current_price * 100)) / day_len_num) * 365)
                    days_call.append(day_len_num)

                else:
                    pass

            except Exception as err:
                print(err)
                print("EEEEEEEEEERRRRRRRROOOOOORRRRR")
                pass


2021-02-19-m
2021-02-26-w
2021-03-05-w
2021-03-12-w
2021-03-19-m


Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.detachFromTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.detachFromTarget: Target closed.
Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error (Target.sendMessageToTarget): No session with given id')>
pyppeteer.errors.NetworkError: Protocol error (Target.sendMessageToTarget): No session with given id


2021-03-26-w
2021-04-01-w
2021-04-16-m
2021-06-18-m
2021-07-16-m
2021-09-17-m
2022-01-21-m
2022-06-17-m
2023-01-20-m


In [7]:

put_data = pd.DataFrame(list(zip(dates, ticker_list, current_price_list, cost_put_call, day_len_list, strike_put_list, puts_bid,
                            income_put, income_put_year )),
                  columns=['Дата', 'Тикер', 'Цена покупки', 'Сколько денег нужно на один опцион PUT + CALL', 'Длина опциона в днях',
                           'Strike PUT', 'Премия PUT', 'Доход от премии PUT', 'Доход от премии PUT в % годовых'
                            ]).reset_index(drop=True)


call_data = pd.DataFrame(list(zip(days_call, strike_call_list, call_bid,  income_call, income_call_yaer)),
                  columns=['Длина опциона в днях', 'Strike CALL', 'Премия CALL', 'Доход от премии CALL', 'Доход от премии CALL в % годовых',
                           ]).reset_index(drop=True)

In [8]:
put_row = f'A{next_ticker_row}'

call_row = f'J{next_ticker_row}'

# # запись в таблицу

worksheet.update(put_row, put_data.values.tolist())
worksheet.update(call_row, call_data.values.tolist())

{'spreadsheetId': '1bfNJIgSEo9V5Jww1-EoUh_onba2bGY2LpDVx4aYlPzc',
 'updatedRange': 'Options_calc!J2:N86',
 'updatedRows': 85,
 'updatedColumns': 5,
 'updatedCells': 425}